In [572]:
%%file ast.lp

#const corange = 10.
constant(1..corange).
%input(1..5).


input(X) :- key(X).
input(X) :- constant(X).


id(0..10).

 
rooted(0).
rooted(B) :- rooted(A), edge(A,B).
:- node(B), not rooted(B).



%%% Each edge has 0-2 children and is connected to the root
%%% Childen have greater ids than their parents
0 { edge(A,B):id(B),B>A} 2 :- rooted(A).

%%% Children cant have multiple parents
%%% It might actually be useful to allow this, so not really an AST anymore
%%% :- id(I), not 0 #sum{1,P:edge(P,I)} 1.
          
%%% TODO:
% All subdags contain at least one key item if the head of the tree is an operation
%%%


%%%{op(I,xor,V1^V2)} :- edge(I,C1), edge(I,C2), C1!=C2, op(C1,T1,V1), op(C2,T2,V2).
%%%{op(I,and,V1&V2)} :- edge(I,C1), edge(I,C2), C1!=C2, op(C1,T1,V1), op(C2,T2,V2). 
%%%{op(I,neg,~V1)}    :- edge(I,C1), unary(I), op(C1,T,V1).
%%%{op(I,input,V1)}  :- leaf(I), input(V1).
    
node(I) :- rooted(I).
binary(I) :- node(I), 2{edge(I,C)}2.
unary(I) :- node(I), 1{edge(I,C)}1.
leaf(I) :- node(I), 0{edge(I,C)}0.
      
bin_op(xor;and).
un_op(neg).
null_op(const;key).

type(T) :- bin_op(T).
type(T) :- un_op(T).
type(T) :- null_op(T).
    
%%%Choose one operation for each node
1{op(I, T):bin_op(T)}1 :- binary(I).
1{op(I, T):un_op(T)}1 :- unary(I).
1{op(I, T):null_op(T)}1 :- leaf(I).
    


%side(left;right).
    
%eval(and,)
  
val(K,I,V1&V2) :- op(I,and),edge(I,I1),edge(I,I2),I1 < I2, val(K,I1,V1), val(K,I1,V2).
val(K,I,V1^V2) :- op(I,xor),edge(I,I1),edge(I,I2),I1 < I2, val(K,I1,V1), val(K,I1,V2).
val(K,I,~V1) :- op(I,neg),edge(I,I1), val(K,I1,V1).
val(K,I,V) :- key(K), op(I,const), constant(V).
val(K,I,K) :- key(K), op(I,key).

hash(K,V) :- val(K,0,V).

occupied(V) :- hash(K,V).
collision :- occupied(V), 2 {hash(K,V)}.

:- collision.
    
%%%collision :- key(K1), key(K2), K1!=K2, hash(K1,V), hash(K2,V).
%%%collision :- key(K1), key(K2), 0 = #sum {2**B,v}
    
%%% Visualization rules

#show children(I,N) : node(I), N = #count{C:edge(I,C)}.
#show op_count(I, X) : node(I), X = #count{T:op(I,T),type(T)}.
    
#show edge/2.
#show op/2.
#show node/2.
%#show edge/1.
%#show op/3.
%#show cnt/2.
%#show children/2.

Overwriting ast.lp


I guess the above would make the basic structure of the AST, but still need some way assign values to the leaf nodes and allow those to be evaluated. No obivous way how to do that.

In [573]:
%%file keys.lp

key(0..10).
key(56).
key(342).
key(33).

Writing keys.lp


In [574]:
!clingo ast.lp keys.lp 2

clingo version 5.2.0
Reading from ast.lp ...
ast.lp:85:1-14: info: no atoms over signature occur in program:
  node/2

Solving...
Answer: 1
edge(0,10) children(10,0) op(10,key) op(0,neg) children(0,1) op_count(0,1) op_count(10,1)
Answer: 2
edge(0,9) op(9,key) op(0,neg) children(0,1) op_count(0,1) children(9,0) op_count(9,1)
SATISFIABLE

Models       : 2+
Calls        : 1
Time         : 11.717s (Solving: 0.38s 1st Model: 0.38s Unsat: 0.00s)
CPU Time     : 11.673s


# Everything below is garbage

In [ ]:
!clingo items.lp perfect.lp

clingo version 5.2.0
Reading from items.lp ...


In [187]:
%%file genhash.lp

#const corange = 10.
constant(1..corange).
input(1..5).


idrange(0..10).



%%% piece(id,child_id_1,child_id_2).
2 {piece(A,B,C):idrange(A),idrange(B),idrange(C)}.

%%% At most one piece per id
:- idrange(A), not 0 #sum{1,B,C:piece(A,B,C)} 1.
        
%%% Piece appears at most once as a child
:- idrange(P), not 0 #sum{1,A,C,left:piece(A,P,C); 1,A,B,right:piece(A,B,P)} 1.
    
%%% Piece can not be its own child
:- idrange(P), piece(P,P,_).
:- idrange(P), piece(P,_,P).
    
%%% If a piece appears as a child, it exists
%TODO
    

    
%:- idrange(B), not 0 #sum{1,A,C:piece(A,B,C)} 1.
%:- idrange(C), not 0 #sum{1,A,B:piece(A,B,C)} 1.

%cnt(X, A) :- idrange(A), X = #sum{1,B,C:piece(A,B,C)}.
    
    

Overwriting genhash.lp


In [21]:
%%file perfect.lp

%%% Attempts to perfectly hash keys given as item(number)

#const corange = 10.
constant(1..corange).
    
%%%operation(type, item, in1, in2, output)
operation(xor, I, A, B, A^B) :- item(I), output(I,A), output(I,B).       
operation(and, I, A, B, A&B) :- item(I), output(I,A), output(I,B). 
operation(or , I, A, B, A?B) :- item(I), output(I,A), output(I,B). 
    

hash(I,X) :- item(I), input(I,V1), input(I,V2), operation(T,I,V1,V2,X).
    
    
output(I,I) :- item(I).
output(I,X) :- item(I), constant(X).
output(I,X) :- item(I), output(I,V1), output(I,V2), operation(T,I,V1,V2,X).
    
    
:- item(A), item(B), A != B, hash(A,X), hash(B,X). 
    
    
    
%%% input constant or item
input(I) :- item(I).
input(I) :- constant(I).
    
    
%:- item(A), item(B), operation(T,ID,) 
    
    
%%output(I,X) :- input(I), operation(T, )
    




Overwriting perfect.lp


In [12]:
%%file minimal.lp

%%% Minimize the range of hashed values
maxhash(X) :- coef(A), mod(M), exp(E), X = #max{ @axbmodc(A,I,E,M):item(I) }.
minhash(X) :- coef(A), mod(M), exp(E), X = #min{ @axbmodc(A,I,E,M):item(I) }.
diff(X) :- maxhash(A), minhash(B), X = A-B.
#minimize{ X:diff(X) }.

Overwriting minimal.lp


In [45]:
%%file ordered.lp

%%% Reject if items are out of order when hashed
:- item(I1), item(I2), I1 < I2, coef(A), mod(M),  A*I1\M > A*I2\M.
:- #true.

Overwriting ordered.lp


In [186]:
!clingo genhash.lp

clingo version 5.2.0
Reading from genhash.lp
Solving...
Answer: 1
constant(1) constant(2) constant(3) constant(4) constant(5) constant(6) constant(7) constant(8) constant(9) constant(10) input(1) input(2) input(3) input(4) input(5) idrange(0) idrange(1) idrange(2) idrange(3) idrange(4) idrange(5) idrange(6) idrange(7) idrange(8) idrange(9) idrange(10) piece(0,7,10) piece(7,0,1)
SATISFIABLE

Models       : 1+
Calls        : 1
Time         : 0.437s (Solving: 0.01s 1st Model: 0.01s Unsat: 0.00s)
CPU Time     : 0.430s


In [38]:
%%file process.py

import json
import sys
import re


result = json.load(sys.stdin)
atoms = result['Call'][0]['Witnesses'][0]['Value']

items = []
modulus = 0
coef = 0
exponent = 0

for a in atoms:
    match = re.match('item\(([0-9]*)\)', a)
    if match:
        items += [int(match.group(1))]
    
    match = re.match('mod\(([0-9]*)\)', a)
    if match:
        modulus = int(match.group(1))

    match = re.match('exp\(([0-9]*)\)', a)
    if match:
        exponent = int(match.group(1))
        
    match = re.match('coef\(([0-9]*)\)', a)
    if match:
        coef = int(match.group(1))

        
table = [(item*coef**exponent%modulus, item) for item in items]
table.sort()

print "Hash function: %dx^%d mod %d" %(coef,exponent,modulus)

print "%7s %7s" % ("hash", "item")
for item in table:
    print "%7d %7d" % item

Overwriting process.py


In [69]:
!clingo items.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 49x^1 mod 36
   hash    item
      3       3
      4     100
      6       6
      7      67
      9       9
     10   34234
     11      23
     13       1
     15     123
     16       4
     19       7
     22      10
     26       2
     29       5
     31      55
     32       8
     35   23123


In [12]:
%%file gen.py
import random

seen = {}
i = 0
while i < 5:
    x = random.randint(10,100)
    if x not in seen:
        print "item(%d)." % (x)
        seen[x] = 1
        i+=1

Overwriting gen.py


In [154]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp ordered.lp quiet.lp --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 55x mod 57
   hash    item
     25      16
     38      38
     44     149
     45     291
     49     631


In [67]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 24x^1 mod 11
   hash    item
      3      62
      4      90
      5      96
      6      91
      8      81
